Demonstrates how to use `InferredField` helper to automatically infer Pydantic fields from single Django model fields.

This is useful if you need to manually create Pydantic models or Pydantic fields from Django model field details.

Especially useful for example with custom Django Ninja endpoints when setting up input values:
```python
from ninja import NinjaAPI
from django2pydantic import InferredField
from django.db.models import Model

api = NinjaAPI()

class MyDjangoItemModel(Model):
    id = models.AutoField(primary_key=True, help_text="The unique identifier for the item")
    ...

@api.get("/items/{id}")
def read_item(request, id: InferredField[MyDjangoItemModel.id]):
    ...
```


Lets proceed to the example usage of the InferredField helper. Lets first setup Django.

In [6]:
# Set up the Django environment for the example
from examples.django_setup import setup_django_environment

setup_django_environment()

Assuming the following Django model with the `id` field as required:

In [7]:
from typing import final

from django.db import models

# Defining some constants so we can assert them later
NAME_VERBOSE="Name of the item"
DESCRIPTION_VERBOSE="Description of the item"

@final
class SomeModel(models.Model):
    id = models.AutoField(primary_key=True, null=False) # Required field here
    name = models.CharField(max_length=100, verbose_name=NAME_VERBOSE)
    details = models.TextField(verbose_name="something....")

    @final
    class Meta:
        app_label = "tests"


/opt/venv/lib/python3.12/site-packages/django/db/models/base.py:368: RuntimeWarning: Model 'tests.somemodel' was already registered. Reloading models is not advised as it can lead to inconsistencies, most notably with related models.
  new_class._meta.apps.register_model(new_class._meta.app_label, new_class)


Assuming the following Pydantic model:

In [8]:
from pydantic import BaseModel

from django2pydantic import InferExcept, InferredField


class SomeSchema(BaseModel):
    id: InferredField[SomeModel.id]
    name: InferredField[SomeModel.name]
    details: InferredField[SomeModel.details, InferExcept(description=DESCRIPTION_VERBOSE)]  # Here we can use the InferExcept helper to modify the inferred field details if we want to overwrite some of the inferred values

The following schema will be produced:

In [9]:
from rich import print_json

schema = SomeSchema.model_json_schema()
print_json(data=schema)


{
  "properties": {
    "id": {
      "description": "id",
      "maximum": 9223372036854775807,
      "minimum": -9223372036854775808,
      "title": "id",
      "type": "integer"
    },
    "name": {
      "description": "Name of the item",
      "maxLength": 100,
      "minLength": 1,
      "title": "Name of the item",
      "type": "string"
    },
    "details": {
      "description": "Description of the item",
      "minLength": 1,
      "title": "something....",
      "type": "string"
    }
  },
  "required": [
    "id",
    "name",
    "details"
  ],
  "title": "SomeSchema",
  "type": "object"
}

As a sanity check, lets now do some asserts to check the generated schema is as intended:

In [10]:
assert schema["properties"]["id"]["type"] == "integer"
assert schema["properties"]["name"]["type"] == "string"
assert schema["properties"]["details"]["type"] == "string"
assert schema["properties"]["name"]["description"] == NAME_VERBOSE
assert schema["properties"]["details"]["description"] == DESCRIPTION_VERBOSE
